In [3]:
import scipy.io
#import entropy
import numpy as np
from scipy.ndimage import zoom
#from skimage.transform import resize
#import cv2
import matplotlib.pyplot as plt
import os
from Utils import eulerian_to_cartesian

In [4]:
dataset_dir='/media/Blue2TB1/Jin_22'

sampled_data_dir=os.path.join(dataset_dir,'sampled_dataset')
NUM_TILES_WIDTH_TRUE_SAL = 256
NUM_TILES_HEIGHT_TRUE_SAL = 256
OUTPUT_TRUE_SALIENCY_FOLDER = os.path.join(dataset_dir,'true_saliency')
trajec_info_dir=os.path.join(dataset_dir,'video_data')

In [5]:
from pathlib import Path

In [42]:
test=np.load(os.path.join(trajec_info_dir,'video_2','video_2_unit_vectors.'))

In [2]:
def get_video_ids(folder):
    list_of_videos = [o for o in os.listdir(folder) if not o.endswith('.gitkeep')]
    return list_of_videos

def get_sampled_unit_vectors():
    pass

def create_and_store_true_saliency(sampled_dataset):
    if not os.path.exists(OUTPUT_TRUE_SALIENCY_FOLDER):
        os.makedirs(OUTPUT_TRUE_SALIENCY_FOLDER)

    # Returns an array of size (NUM_TILES_HEIGHT_TRUE_SAL, NUM_TILES_WIDTH_TRUE_SAL) with values between 0 and 1 specifying the probability that a tile is watched by the user
    # We built this function to ensure the model and the groundtruth tile-probabilities are built with the same (or similar) function
    def from_position_to_tile_probability_cartesian(user_data):
        user_num = user_data.shape[0]
        yaw_grid, pitch_grid = np.meshgrid(np.linspace(0, 1, NUM_TILES_WIDTH_TRUE_SAL, endpoint=False),
                                           np.linspace(0, 1, NUM_TILES_HEIGHT_TRUE_SAL, endpoint=False))
        yaw_grid += 1.0 / (2.0 * NUM_TILES_WIDTH_TRUE_SAL)
        pitch_grid += 1.0 / (2.0 * NUM_TILES_HEIGHT_TRUE_SAL)
        yaw_grid = yaw_grid * 2 * np.pi
        pitch_grid = pitch_grid * np.pi
        x_grid, y_grid, z_grid = eulerian_to_cartesian(theta=yaw_grid, phi=pitch_grid)
        x_grid = np.expand_dims(x_grid, axis=0)
        y_grid = np.expand_dims(y_grid, axis=0)
        z_grid = np.expand_dims(z_grid, axis=0)
        x_pos = np.expand_dims(user_data[:, 0], axis=(1, 2))
        y_pos = np.expand_dims(user_data[:, 1], axis=(1, 2))
        z_pos = np.expand_dims(user_data[:, 2], axis=(1, 2))
        dot_product = x_grid * x_pos + y_grid * y_pos + z_grid * z_pos
        great_circle_distance = np.arccos(np.clip(dot_product, -1.0, 1.0))
        gaussian_orth = np.exp((-1.0 / (2.0 * np.square(0.1))) * np.square(great_circle_distance))
        return gaussian_orth

    videos = get_video_ids(sampled_dataset)
    for enum_video, video in enumerate(videos):
        print('creating true saliency for video', video, '-', enum_video+1, '/', len(videos))
        real_saliency_for_video = []
        video_data=np.load(os.path.join(sampled_dataset,video,f'{video}_unit_vectors.npy'))
        max_num_samples=video_data.shape[1]
        for x_i in range(max_num_samples):
            tileprobs_for_video_cartesian=from_position_to_tile_probability_cartesian(video_data[:,x_i,:])
            real_saliency_cartesian = np.sum(tileprobs_for_video_cartesian, axis=0) / tileprobs_for_video_cartesian.shape[0]
            real_saliency_for_video.append(real_saliency_cartesian)
        real_saliency_for_video = np.array(real_saliency_for_video)

        true_sal_out_file = os.path.join(OUTPUT_TRUE_SALIENCY_FOLDER, video)
        np.save(true_sal_out_file, real_saliency_for_video)

In [103]:
#create_and_store_true_saliency(trajec_info_dir)

creating true saliency for video video_1 - 1 / 27
creating true saliency for video video_2 - 2 / 27
creating true saliency for video video_3 - 3 / 27
creating true saliency for video video_4 - 4 / 27
creating true saliency for video video_5 - 5 / 27
creating true saliency for video video_6 - 6 / 27
creating true saliency for video video_7 - 7 / 27
creating true saliency for video video_8 - 8 / 27
creating true saliency for video video_9 - 9 / 27
creating true saliency for video video_10 - 10 / 27
creating true saliency for video video_11 - 11 / 27
creating true saliency for video video_12 - 12 / 27
creating true saliency for video video_13 - 13 / 27
creating true saliency for video video_14 - 14 / 27
creating true saliency for video video_15 - 15 / 27
creating true saliency for video video_16 - 16 / 27
creating true saliency for video video_17 - 17 / 27
creating true saliency for video video_18 - 18 / 27
creating true saliency for video video_19 - 19 / 27
creating true saliency for vid

In [11]:
def shannon_entropy(saliency_map):
    saliency_map_squared=np.square(saliency_map)
    c_entropy=-np.sum(saliency_map_squared*np.log(saliency_map_squared))
    return c_entropy

def get_vid_shannon_entropy(saliency_maps):
    epsilon=np.finfo(float).eps
    squared_maps=np.square(saliency_maps)
    squared_maps=np.where(squared_maps==0,epsilon,squared_maps)
    c_entropy=-np.sum(squared_maps*np.log(squared_maps),axis=1)
    return c_entropy

def get_content_entropies(saliency_folder, trajec_info_dir):
    saliencies=os.listdir(saliency_folder)
    saliencies=['video_20.npy']
    
    for vid in saliencies:
        vid_name=vid.split('.')[0]
        print(f'Processing {vid_name}')
        vid_saliency=np.load(os.path.join(saliency_folder,vid))
        dest_path=os.path.join(trajec_info_dir,vid_name)
        num_maps=len(vid_saliency)
        c_entropies=get_vid_shannon_entropy(vid_saliency.reshape(num_maps,-1))
        print(np.mean(c_entropies))
        #np.save(os.path.join(dest_path,f'{vid_name}_content_entropy.npy'),c_entropies)
            

In [12]:
sal_folder=OUTPUT_TRUE_SALIENCY_FOLDER
op_folder=trajec_info_dir
get_content_entropies(sal_folder,op_folder)

Processing video_20
386.2108866946764


In [142]:
sal_folder='Xu_CVPR_18/true_saliency'
op_folder='D:/CVPR18/Trajectory_info'
get_content_entropies(sal_folder,op_folder)

In [148]:
sal_folder='Fan_NOSSDAV_17/true_saliency'
op_folder='D:/Fan_NOSSDAV_17/video_data'
get_content_entropies(sal_folder,op_folder)

In [6]:
sal_folder='D:/PAMI18/true_saliency'
op_folder='D:/PAMI18/video_data'
get_content_entropies(sal_folder,op_folder)

Processing A380
Processing AcerEngine
Processing AcerPredator
Processing AirShow
Processing BFG
Processing BTSRun
Processing Bicycle
Processing BlueWorld
Processing CMLauncher
Processing CMLauncher2
Processing CS
Processing Camping
Processing CandyCarnival
Processing Castle
Processing Catwalks
Processing DanceInTurn
Processing Dancing
Processing DrivingInAlps
Processing Egypt
Processing F5Fighter
Processing Flight
Processing GTA
Processing GalaxyOnFire
Processing Graffiti
Processing Guitar
Processing HondaF1
Processing IRobot
Processing InsideCar
Processing KasabianLive
Processing KingKong
Processing Lion
Processing LoopUniverse
Processing MC
Processing Manhattan
Processing MercedesBenz
Processing Motorbike
Processing Murder
Processing NotBeAloneTonight
Processing Orion
Processing Parachuting
Processing Parasailing
Processing Pearl
Processing Predator
Processing ProjectSoul
Processing Rally
Processing RingMan
Processing RioOlympics
Processing Roma
Processing Shark
Processing Skiing
Pro

In [9]:
sal_folder='D:/MMSys18/true_saliency'
op_folder='D:/MMSys18/video_data'
get_content_entropies(sal_folder,op_folder)

Processing 1_PortoRiverside
Processing 2_Diner
Processing 3_PlanEnergyBioLab
Processing 4_Ocean
Processing 5_Waterpark
Processing 6_DroneFlight
Processing 7_GazaFishermen
Processing 8_Sofa
Processing 9_MattSwift
Processing 10_Cows
Processing 11_Abbottsford
Processing 12_TeatroRegioTorino
Processing 13_Fountain
Processing 14_Warship
Processing 15_Cockpit
Processing 16_Turtle
Processing 17_UnderwaterPark
Processing 18_Bar
Processing 19_Touvet
